### Steps for this exercise
##### 1. necessary imports 
##### 2. setting the model parameter and intializing either openai or ollama
##### 3. Writing the Website Utility Class to scrape data even links this time
##### 4. get all the necessary links for the given company's homePage link using the get_all_links(url) function which communicates with ollama using the system and user prompts
##### 5. Then we create the brochure and additionaly stream if the client supports after communication with the ollama api using the system and user prompt.

In [1]:
import requests
import ollama
import json
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import re

In [2]:
MODEL = 'gemma3:4b'
ollama_via_openai = OpenAI(base_url="https://localhost:11434/v1",api_key="ollama")

In [3]:
## A utility class for scraping Websites now also with links

class Website:
    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body,'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n',strip=True)
        else:
            self.text=""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents: \n{self.text}\n"

In [4]:
sec = Website("https://github.com/lostcoin-afk")
sec.links

['#start-of-content',
 '/',
 '/login?return_to=https%3A%2F%2Fgithub.com%2Flostcoin-afk',
 'https://github.com/features/copilot',
 'https://github.com/features/spark',
 'https://github.com/features/models',
 'https://github.com/security/advanced-security',
 'https://github.com/features/actions',
 'https://github.com/features/codespaces',
 'https://github.com/features/issues',
 'https://github.com/features/code-review',
 'https://github.com/features/discussions',
 'https://github.com/features/code-search',
 'https://github.com/why-github',
 'https://docs.github.com',
 'https://skills.github.com',
 'https://github.blog',
 'https://github.com/marketplace',
 'https://github.com/mcp',
 'https://github.com/features',
 'https://github.com/enterprise',
 'https://github.com/team',
 'https://github.com/enterprise/startups',
 '/solutions/industry/nonprofits',
 '/solutions/use-case/app-modernization',
 '/solutions/use-case/devsecops',
 '/solutions/use-case/devops',
 '/solutions/use-case/ci-cd',
 '/

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON object format as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms o f Service, Privacy, email links \n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
def get_links(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role":"system","content":link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ]
    )
    result = response["message"]["content"]

    cleaned_result = re.sub(r"^```(?:json)?\n|\n```$", "", result.strip())

    return json.loads(cleaned_result)

In [8]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'models', 'url': 'https://huggingface.co/models/'},
  {'type': 'datasets', 'url': 'https://huggingface.co/datasets/'},
  {'type': 'spaces', 'url': 'https://huggingface.co/spaces/'},
  {'type': 'docs', 'url': 'https://huggingface.co/docs/'},
  {'type': 'models', 'url': 'https://huggingface.co/models/'},
  {'type': 'datasets', 'url': 'https://huggingface.co/datasets/'},
  {'type': 'spaces', 'url': 'https://huggingface.co/spaces/'},
  {'type': 'docs', 'url': 'https://huggingface.co/docs/'}]}

#### Now we have all the links in the given website/companyLink. We will now move on to the next step of actually creating the Brochure.

In [9]:
def get_all_details(url):
    result = "Landing Page"
    result += Website(url).get_contents()
    links = get_links(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [10]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [11]:
def get_brochure_user_prompt(company_name,url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5000] # Truncate if more than 5000 characters
    return user_prompt

In [12]:
def create_brochure(companyName,url):
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role":"system","content":system_prompt},
            {"role":"user","content":get_brochure_user_prompt(companyName,url)}
        ],
    )   
    display(Markdown(response["message"]["content"])) 

In [13]:
create_brochure("satyasai","https://github.com/lostcoin-afk")

Okay, here's a brochure based on the information gathered from the GitHub pages provided for lostcoin-afk. It’s designed to be adaptable for different audiences – customers, investors, and recruits.

---

**Brochure: lostcoin-afk – Empowering Innovation Through Developer Tools**

**(Front Panel - Image: A dynamic graphic representing code, collaboration, and AI)**

**Headline:** Unlock Your Potential with the AI-Powered Developer Platform

**Subheading:**  lostcoin-afk is revolutionizing software development with cutting-edge tools and a community-focused approach.

**(Inside Left Panel - Focus: Customer Solutions)**

**Headline:  Accelerate Your Development Journey**

**What is lostcoin-afk?**

lostcoin-afk is a comprehensive developer platform built on GitHub, designed to streamline the entire software development lifecycle.  We provide the tools you need to build, test, secure, and deploy your applications faster and more efficiently. 

**Key Features for Developers:**

*   **AI-Powered Assistance:**  Leverage GitHub Copilot to write code faster and with greater accuracy.
*   **DevSecOps Support:** Integrate security practices throughout your development workflow using GitHub Advanced Security.
*   **Automated Workflows:**  Utilize Actions to automate repetitive tasks and ensure consistent processes.
*   **Rapid Development Environments:**  Instant dev environments with Codespaces for maximum productivity.
*   **Collaboration Tools:** Seamlessly collaborate on code, track issues, and manage code changes with integrated features.

**Solutions for Your Business:**

*   **App Modernization:**  Migrate legacy applications to modern frameworks.
*   **DevOps & CI/CD:** Automate your continuous integration and continuous delivery pipelines.
*   **Security Focused:** Protect your applications from vulnerabilities with advanced security features.

**(Inside Right Panel - Focus: Investor & Recruit Information)**

**Headline: Join the Future of Software Development**

**Why Invest in lostcoin-afk?**

*   **Rapid Growth:** The demand for developer tools is booming, and lostcoin-afk is positioned for significant growth within the GitHub ecosystem.
*   **Innovation:**  We are committed to investing in cutting-edge AI and DevOps technologies.
*   **Community Driven:**  Our strong community and open-source focus provide a sustainable competitive advantage.
*   **Strategic Partnerships:** Expanding our integrations through the GitHub Marketplace.


**Careers at lostcoin-afk:**

**We’re Hiring!**  We are looking for talented and passionate individuals to join our team. We're a fast-paced, collaborative environment where you'll have the opportunity to make a real impact.
*   **Software Engineers:**  Develop and maintain our core platform.
*   **DevOps Engineers:** Optimize our development and deployment processes.
*   **Security Engineers:**  Protect our platform and our users.

**Resources:** [GitHub Links Listed Here - Documentation, Blog, Events, etc.]

**(Back Panel - Image:  GitHub Logo)**

**Contact Us:**  [GitHub Support Link]
**Website:** [GitHub Link]
**Follow Us:**  [Social Media Links - Placeholder]

---

**Notes and Considerations:**

*   **Dynamic Content:** This brochure is a starting point.  The GitHub pages are frequently updated, so it's essential to keep this brochure current.
*   **Visuals:** The addition of relevant images and graphics will significantly enhance the brochure's appeal.
*   **Targeted Messaging:** Adapt the brochure's tone and focus depending on the specific audience (e.g., more technical detail for recruits, more financial highlights for investors).

Let me know if you'd like me to refine this brochure further or focus on a specific aspect (e.g., creating a version specifically for investors).

In [14]:


from IPython.display import display, Markdown

def create_stream_brochure(company_name, url):
    stream = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True  # Only if your Ollama client supports this
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        # Adjust this depending on Ollama's streaming format
        delta = chunk.get("message", {}).get("content", "")
        response += delta or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
create_stream_brochure("satyasai","https://www.satyasai.co.in")

Okay, here's a brochure designed using the information provided about Satyasai Construction. I've aimed for a professional and concise look, suitable for potential customers, investors, and recruits.

---

## **Satyasai Construction: Delivering Efficient Materials Handling Solutions**

**(Brochure Front - Image: A dynamic image of equipment in action – ideally a conveyor system or material handling rig)**

**Headline:**  Reliable. Innovative. Results-Driven Materials Handling

---

**Inside Left Panel – For Customers & Investors**

**Who We Are:**

Satyasai Engineering & Construction is a leading materials handling and construction services provider, dedicated to delivering seamless, efficient, and reliable solutions to the construction industry. With over 15 years of experience, we specialize in the loading, unloading, movement, transportation, and management of materials and equipment on complex construction sites.

**Our Mission:**

To revolutionize the construction industry by providing innovative solutions for the movement, transportation, and management of materials and equipment.

**Key Services:**

* **Facility Design & Construction:**  Custom facility design and construction tailored to optimize workflow.
* **Conveyor Systems & Automation:**  Advanced solutions to enhance material handling efficiency.
* **Equipment Installation & Maintenance:**  Ensuring long-term reliability and performance.
* **System Upgrades & Retrofitting:** Modernizing existing systems for improved performance.
* **Waste Removal & Recycling:** Streamlined waste management solutions.

**Client Testimonials:**

* "Satyasai Engineering & Construction Has Solid Strength with Sufficient Handling Equipment, Tools and Tackles and Well Proffessional Manpower." - Indian Oil Corporation Limited – Refinery Division
* "Satyasai Engineering & Construction is proffesional Materials Handling Company. We have work with them a long time more than 8 years with fully sattisfiction." - Engineers India Limited - Project Consultant
* "National Aluminium Corporation Limited - Aluminium Refinery” – Quality and Safety is the asset of M/S Satyasai Engineering & Construction with strong and Proffessional Management Team and friendly service provider. 


**Our Equipment:**

* Haldia Refinery
* Paradeep Refinery
* Vizag Refinery
* NALCO
* Panipat Refinery

**(Image: A smaller image showcasing a variety of equipment)**


---

**Inside Right Panel – For Recruits & Investors**

**Company Culture & Values:**

At Satyasai, we pride ourselves on a culture of:

* **Quality & Safety:** Our top priority is the safety of our team and the quality of our work.
* **Innovation:** We constantly seek new technologies and methods to improve our services.
* **Teamwork:** We foster a collaborative environment where everyone contributes to success.
* **Results-Oriented:** We are committed to delivering exceptional results for our clients.

**Meet Our Team:**

* **Active Experts:** 450+ Skilled Professionals
* **Satisfied Clients:** 670+  Clients Served
* **Project Complete:** 1500+ Projects Completed

**Careers with Satyasai:**

We're growing and seeking talented individuals to join our team.  If you are passionate about materials handling, construction, or engineering, we encourage you to explore opportunities with us.  (Link to a hypothetical careers page – e.g., “Learn More at [Website Address]”)

**Contact Us:**

* **Head Office:** Chunabellari, PO: Paradeepgarh, Dist.Jagatsinghapur, Odisha, 754141
* **Corporate Office:** Haldia Refinery, PO: Haldia Refinery Campus, PS: Haldia, West Bengal, 721602
* **Email:** admin@satyasai.co.in
* **Phone:** +91 99384 73499 / +91 90400 73496

**(Image: A team photo – representing a diverse and skilled workforce)**


---

**Back Panel – Company Information & Branding**

**(Logo: Satyasai Engineering & Construction Logo)**

**Satyasai Engineering & Construction** – Delivering the Power of Efficient Materials Handling.

**Website:** [Insert Website Address Here]
**Social Media:** (Links to social media profiles if available)

© 2024 Satyasai Engineering & Construction. 

---

**Notes & Considerations:**

*   **Images:** The brochure would greatly benefit from high-quality images of the equipment, construction sites, and team members.
*   **Website Address:** Replace "[Insert Website Address Here]" with the actual website address.
*   **Branding:** Ensure consistent use of the Satyasai logo and color scheme throughout.
*   **Distribution:** This brochure is designed for print, but could also be adapted for digital distribution.

Let me know if you'd like me to refine any aspect of this brochure!